# GSMArena Scraper

### Requisitos:
- Instalar las dependencias necesarias:  

In [1]:
# %pip install pandas requests beautifulsoup4 selenium

In [2]:
import sys
import os

# Subir un nivel (de notebooks/ a raiz/)
sys.path.append(os.path.abspath("..")) 
from scraper.base import GSMArenaScraper

In [3]:
scraper = GSMArenaScraper(base_url="https://www.gsmarena.com/makers.php3", 
                          pause_every=50, 
                          )
scraper.say_hello()
scraper.initial_scraper()
scraper.brand_scraper()

2025-09-08 19:12:44 [INFO] GSMArenaScraper: 👋 Hola desde GSMArenaScraper
2025-09-08 19:12:44 [INFO] GSMArenaScraper: Iniciando scraping de https://www.gsmarena.com/makers.php3
2025-09-08 19:12:45 [INFO] GSMArenaScraper: Scraping inicial terminado. Se encontraron 125 marcas.
2025-09-08 19:12:45 [INFO] GSMArenaScraper: Iniciando scraping de las 125 marcas
2025-09-08 19:12:49 [INFO] GSMArenaScraper: Visitando la marca Acer
2025-09-08 19:13:02 [INFO] GSMArenaScraper: Visitando la marca alcatel
2025-09-08 19:13:22 [INFO] GSMArenaScraper: Visitando la marca Allview


In [4]:
scraper.devices[0]

{'brand': 'Acer',
 'name': 'Iconia A14',
 'device_url': 'https://www.gsmarena.com/acer_iconia_a14-14120.php',
 'specs': []}

In [5]:
scraper.device_scraper()

2025-09-08 19:13:44 [INFO] GSMArenaScraper: Iniciando scraping de los equipos
2025-09-08 19:13:48 [INFO] GSMArenaScraper: Visitando el equipo Iconia A14
2025-09-08 19:14:38 [INFO] GSMArenaScraper: Visitando el equipo Iconia A16
2025-09-08 19:14:56 [INFO] GSMArenaScraper: Visitando el equipo Iconia X14


In [6]:
scraper.devices

[{'brand': 'Acer',
  'name': 'Iconia A14',
  'device_url': 'https://www.gsmarena.com/acer_iconia_a14-14120.php',
  'specs': {'Technology': 'No cellular connectivity',
   '2G bands': 'N/A',
   '3G bands': 'N/A',
   '4G bands': 'N/A',
   'GPRS': 'No',
   'EDGE': 'No',
   'Announced': '2025, September 03',
   'Status': 'Coming soon. Exp. release 2025, November',
   'Dimensions': '324.6 x 211 x 8 mm (12.78 x 8.31 x 0.31 in)',
   'Weight': '910 g (2.01 lb)',
   'SIM': 'No',
   '': 'UFS 3.1',
   'Type': '8000 mAh',
   'Size': '14.0 inches, 568.3 cm2 (~83.0% screen-to-body ratio)',
   'Resolution': '1200 x 1920 pixels, 16:10 ratio (~162 ppi density)',
   'OS': 'Android 15',
   'Chipset': 'Allwinner A733 (12 nm)',
   'CPU': 'Octa-core (2x2.0 GHz Cortex-A76 & 6x1.8 GHz Cortex-A55)',
   'GPU': 'IMG BXM-4-64 MC01',
   'Card slot': 'microSDXC (dedicated slot)',
   'Internal': '256GB 8GB RAM',
   'Single': '5 MP',
   'Features': 'LED flash',
   'Video': 'Yes',
   'Loudspeaker': 'Yes, with stereo sp